In [206]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [214]:
historical_data_path = "data/synthetic_historical_report.csv"
current_data_path = "data/synthetic_current_report.csv"

train = pd.read_csv(historical_data_path)

In [215]:
train.head()

,Transaction ID,Date,Account Number,Bank Name,Bank Statement Amount,Book Records Amount,Match Status,COMMENT
0,TXN-006253,2040-02-13,ACCT-611080,Bank of America,-4.003063,-4.087139,Break,Bank Error
1,TXN-004685,2035-10-29,ACCT-522365,Citibank,301.600500,458.671660,Match,NaN
2,TXN-001732,2027-09-28,ACCT-951048,Wells Fargo,2459.547900,2346.492700,Match,NaN
3,TXN-004743,2035-12-26,ACCT-205600,Wells Fargo,99.653336,68.275390,Match,NaN
4,TXN-004522,2035-05-19,ACCT-308181,Wells Fargo,-3.955013,-3.947105,Break,Missing in Books


In [208]:
# Preprocessing
# -------------
# Drop Transaction ID (not useful for modeling)
df = train.drop("Transaction ID", axis=1)

# Convert Date to datetime and extract features
df["Date"] = pd.to_datetime(df["Date"])
df["DayOfWeek"] = df["Date"].dt.dayofweek  # 0=Monday, 6=Sunday
df["Month"] = df["Date"].dt.month
df = df.drop("Date", axis=1)  # Drop original date column

# Feature Engineering: Calculate Amount Difference
df["Amount Difference"] = df["Bank Statement Amount"] - df["Book Records Amount"]
df["Absolute Difference"] = np.abs(df["Amount Difference"])

# Encode Categorical Features
categorical_features = ["Bank Name", "Match Status"]
numerical_features = ["Bank Statement Amount", "Book Records Amount", "Amount Difference", "Absolute Difference", "DayOfWeek", "Month"]

# Preprocessing Pipeline
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numerical_features),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_features)
    ])

# Target Variable
y = df["COMMENT"]
X = df.drop("COMMENT", axis=1)

# Split Data (time-based splitting would be better in practice)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build Pipeline
pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("classifier", KNeighborsClassifier(n_neighbors=3))  # Adjust k here
])

In [209]:
# Train Model
pipeline.fit(X_train, y_train)

# Predict & Evaluate
y_pred = pipeline.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

ValueError: Input contains NaN

In [199]:
# Predict New Data
# ----------------
test = pd.read_csv(current_data_path)
# Reuse the same pipeline for preprocessing and prediction
new_pred = pipeline.predict(test)
print("\nPredicted COMMENT for new transaction:", new_pred[0])

Device set to use cpu


In [200]:
sample_record = pd.read_csv(current_data_path).iloc[0].to_dict()
sample_record

{'Transaction ID': 'TXN-000002',
 'Date': '2023-01-02',
 'Account Number': 'ACCT-697028',
 'Bank Name': 'HSBC',
 'Bank Statement Amount': 0.23801884,
 'Book Records Amount': 0.85056293,
 'Match Status': 'Break'}

In [204]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

class EnhancedReconciliationSystem:
    def __init__(self, historical_data_path, llm):
        self.llm = llm
        self.encoder = SentenceTransformer('models/embeddings/all-MiniLM-L6-v2')
        self.historical_data = self._load_historical_data(historical_data_path)
        self._build_resolution_patterns()
        self._initialize_llm_chain()

    def _load_historical_data(self, path):
        """Load and process historical data"""
        df = pd.read_csv(path).head(100)
        
        # Clean and preprocess comments
        df['COMMENT'] = df['COMMENT'].fillna('').apply(lambda x: x.lower().strip())
        
        # Create embeddings for semantic matching
        df['comment_embedding'] = list(self.encoder.encode(df['COMMENT'].tolist()))
        
        # Derive resolution status from comments
        df['resolved'] = df.apply(lambda x: self._derive_resolution_status(x), axis=1)
        
        return df

    def _derive_resolution_status(self, row):
        """Determine resolution status from match status and comments"""
        if row['Match Status'] == 'Break':
            # Consider empty comment as resolved (could be auto-resolved) or unresolved based on business logic
            return 0 if row['COMMENT'] == '' else 1  # 0=unresolved, 1=resolved
        return 1  # Non-break status considered resolved

    def _build_resolution_patterns(self):
        """Create resolution pattern vectors"""
        self.resolution_patterns = {
            'rounding': self.encoder.encode("Factor rounding is causing minor delta"),
            'false_break': self.encoder.encode("false break due to additional line item"),
            'var_updates': self.encoder.encode("periodic VAR updates ONLY in backend"),
            'system_sync': self.encoder.encode("systems are in line")
        }

    def _initialize_llm_chain(self):
        """Set up LLM for comment analysis"""
        self.prompt_template = PromptTemplate(
            input_variables=["comment", "history"],
            template="""Analyze reconciliation comment and historical patterns:
Current Comment: {comment}
Historical Resolutions: {history}

Categorize into:
- Rounding Issue
- System Sync Delay  
- Data Entry Error
- Valid Break
- Other

Return only the category."""
        )
        self.llm_chain = LLMChain(llm=self.llm, prompt=self.prompt_template)

    def _match_historical_pattern(self, comment):
        """Find similar historical resolutions using cosine similarity"""
        if not comment:
            return 'Historically Resolved'

        emb = self.encoder.encode(str(comment))
        similarities = {
            k: cosine_similarity([emb], [v])[0][0] 
            for k, v in self.resolution_patterns.items()
        }
        max_key = max(similarities, key=similarities.get)
        return max_key if similarities[max_key] > 0.7 else 'Other'

    def _categorize_break(self, merged_row):
        """Categorize break using hybrid approach"""
        if merged_row['Match Status'] != 'Break':
            return 'Match - No Action'
            
        # Check for empty comment in break case
        if not merged_row['COMMENT']:
            return 'Auto-Resolved - Historical Pattern'
            
        # Match historical patterns
        pattern = self._match_historical_pattern(merged_row['COMMENT'])
        
        # Use LLM categorization for novel patterns
        if pattern == 'Other':
            # Sample a few historical comments for context (if available)
            history_samples = self.historical_data['COMMENT'].dropna().sample(min(3, len(self.historical_data))).tolist()
            return self.llm_chain.run({
                'comment': merged_row['COMMENT'],
                'history': history_samples
            })
            
        return pattern

    def _recommend_action(self, category):
        """Get recommended action based on category"""
        action_map = {
            'Rounding Issue': 'No action needed - Auto resolve',
            'System Sync Delay': 'Monitor for 7 days',
            'Data Entry Error': 'Create correction ticket',
            'Valid Break': 'Escalate to operations team',
            'Historically Resolved': 'Close break',
            'Auto-Resolved - Historical Pattern': 'Close break',
            'Match - No Action': 'No action needed',
            'Other': 'Needs manual review'
        }
        return action_map.get(category, 'Needs manual review')

    def _calculate_confidence(self, category, merged_data):
        """Calculate confidence score based on historical matches"""
        if category in ['Rounding Issue', 'System Sync Delay']:
            return 0.95
        if len(merged_data) > 0:
            return min(0.9, len(merged_data) / 10)
        return 0.7

    def process_record(self, record):
        """Process an individual record.
           It merges the record with historical data and returns classification details.
        """
        # Merge with historical data using left join to preserve all incoming records
        merged = pd.merge(
            pd.DataFrame([record]), 
            self.historical_data,
            on=['Transaction ID', 'Account Number'],
            how='left',
            suffixes=('', '_hist')
        )
        merged_row = merged.iloc[0]
        
        # Categorize break
        category = self._categorize_break(merged_row)
        
        # Recommend action
        action = self._recommend_action(category)
        
        return {
            'transaction_id': record.get('Transaction ID'),
            'match_status': record.get('Match Status'),
            'break_category': category,
            'recommended_action': action,
            'confidence': self._calculate_confidence(category, merged)
        }

    def process_records(self, df_records):
        """Process a dataframe of records and return classifications for each."""
        results = []
        for idx, record in df_records.iterrows():
            result = self.process_record(record)
            results.append(result)
        return pd.DataFrame(results)


In [205]:
# Example usage:
# llm = YourLLM()  # instantiate your LLM
ers = EnhancedReconciliationSystem(historical_data_path, llm)
new_data = pd.read_csv(current_data_path)
result_df = ers.process_records(new_data)
print(result_df)


KeyboardInterrupt: 

In [ ]:
result_df